# **pip install**
Install essential libraries in correct versions

In [4]:
pip install colour

In [5]:
pip install -U kaleido

     |████████████████████████████████| 79.9 MB 126 kB/s 


In [6]:
pip install orca

     |████████████████████████████████| 244 kB 5.3 MB/s 


In [2]:
pip install plotly==5.3.1


     |████████████████████████████████| 23.9 MB 42.8 MB/s 
  Attempting uninstall: plotly
    Found existing installation: plotly 4.4.1
    Uninstalling plotly-4.4.1:
      Successfully uninstalled plotly-4.4.1


**IMPORT LIBRARIES**

In [7]:
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from colour import Color
import numpy as np
from scipy.integrate import odeint
import matplotlib.pyplot as plt
from math import sqrt
import pandas as pd
from scipy.interpolate import interp1d
from scipy import optimize
from scipy import integrate as sp


import os

if not os.path.exists("images"):
    os.mkdir("images")



# **MODEL**
Ordinary Differential Equations, Parameters and Initial Conditions

In [8]:

#Sporulation probability
def normal_distri(t, alphamax, sig, tmax):
    alpha = alphamax * ((1. / (sqrt(np.pi * 2.) * sig)) * np.exp(-0.5 * (np.power((t - tmax), 2.) / (sqrt(2) * np.power(sig, 2.)))))

    return alpha
#substrate intake
def substr(s0,t, freq):
    s = (s0/2)+ (s0/2)*np.cos(t*freq)
   
    return s

#monod equation
def monod(s, v, Ks):
    monod = ((s * v) / (s + Ks))

    return monod
#Chassis replication
def replication(Bs,s, v, Ks):
    x_growth = monod(s, v, Ks)*Bs

    return x_growth


#Global Transcription Regulation
def ktx(t, Vmaxtx, Km, s0, v, Ks):
    k = (Vmaxtx * (monod(s0,v,Ks)/ Km) )/ ((1 + ((monod(s0,v,Ks)/ Km))))

    return k

#MODEL
def modelkinetic(ics,t, K05Bs, FD, trigger0, v, Ks, y, lamb, alphamax, sig, tmax, R, DEGrna, DEGamp, Kd, ktl, dig_rat, growthLeish, n):
    #v, Ks, y, lamb, alphamax, sig, tmax, R, DEGrna, DEGamp, Kd, ktl, dig_rat, growthLeish, n = parameters
    Bs0, s0, mRNA0, mRNAon0, proAMP0, AMPa0, Leish0 = ics

    #Chassis 
    dBs = replication(Bs0,s0, v, Ks) - (np.log(2)/lamb + normal_distri(t, alphamax, sig, tmax) + ((AMPa0 ** n) / ((K05Bs ** n) + (AMPa0 ** n))))*Bs0
    
    #Substrate 
    ds = substr(0.03,t, freq) - (R+(replication(Bs0,s0, v, Ks))/y ) * s0
    #mRNAtoehold 
    dmRNA = ktx(t, Vmaxtx, Km, s0, v, Ks) * DNA - (DEGrna + monod(s0, v, Ks)+(trigger0 * mRNA0 / Kd)) * mRNA0
    #mRNAon 
    dmRNAon = (trigger0 * mRNA0 / Kd) - (DEGrna) * mRNAon0
    #proAMP
    dproAMP = ktl * mRNAon0 - (DEGamp + monod(s0, v, Ks) + dig_rat) * proAMP0
    #AMPa
    dAMPa = (dig_rat * proAMP0)*Bs0/FD - (DEGamp) * AMPa0
    #Leish
    dLeish = (growthLeish - ((AMPa0 ** n) / ((K05 ** n) + (AMPa0 ** n)))) * Leish0

    return dBs, ds, dmRNA, dmRNAon, dproAMP, dAMPa, dLeish



####PARAMETER VALUES####

#Initial conditions
Bsi = (10**2)  # Bacteria concentration (cell/L)
Si = 0.3    # Substrate concentration (g/L)
Trigger = 100 # Trigger concentration (nM)
mRNA = 0.01   # mRNA inactivated concentration (nM)
mRNAon = 0.01   # mRNA with Toehold activated concentration (nM)
proAMP = 0.01   # proAMP concentration (nM)
AMPa = 0.01   # AMP activated (nM)
Leish1 = 3*10**3  #Leishmania concentration inside the vector (cell/L)

#Bs growth
v = 0.016
Ks = 0.047
y = 6.4*10**4
K05Bs = 20000

#Sporulation probability
alphamax = 0.13 #1/min
sig = 720 #min
tmax = 4464 #min

#Substrate flux
freq = 0.016
lamb = 90
R = 0.030 #min-1

#Transcription
DNA = 1
Km = 0.013 # 1/min - estimation
Vmaxtx = 50 #nMmin-1

#Translation
ktl = 150 #nMmin-1

#Thermodynamics
gas = 8.32 #J.mol-1.K
T = 310 #K
DeltaG0 = -30
Kd = (1/np.exp(DeltaG0/(gas*T)))*60 #!!!!!!!!!

#AMP activation
dig_rat = 0.8

#Degradation
Tm = 3
Tp = 10
DEGrna = np.log(2)/Tm
DEGamp = (np.log(2)/Tp)
FD= 10000

#LeishDeath
n = 2
K05 = 8100 # nM
growthLeish = 0.0016

Parameters = K05Bs, FD,Trigger, v, Ks, y, lamb, alphamax, sig, tmax, R, DEGrna, DEGamp, Kd, ktl, dig_rat, growthLeish, n
P0 = np.array([Bsi, Si, mRNA, mRNAon, proAMP, AMPa, Leish1])

#time-series (min)
t0 = 0
tf = 500
t=np.linspace(t0,tf, tf+1)

ODE_Geral=odeint(modelkinetic,P0, t, args=( K05Bs, FD,Trigger,v, Ks, y, lamb, alphamax, sig, tmax, R, DEGrna, DEGamp, Kd, ktl, dig_rat, growthLeish, n))

#ODE_Geral1 = ODE_Geral.transpose()

BsF = ODE_Geral[:,0]
SubF = ODE_Geral[:,1]
mRNAF = ODE_Geral[:,2]
mRNAonF = ODE_Geral[:,3]
proAMPF = ODE_Geral[:,4]
AMPaF = ODE_Geral[:,5]
LeishF = ODE_Geral[:,6]

#plot information
mRNAon_delay = int(t[np.nonzero(mRNAonF > mRNAonF.max()*0.5)[0][0]])
ymRNAon = mRNAonF[mRNAon_delay]
Leishmin = LeishF.min()
Leishmin_delay = int(t[np.nonzero(LeishF < LeishF[0]*0.5)[0][0]])


# **SIMULATION**

In [ ]:
#color
color = ["#482173", "#2e6f8e", "#29af7f", "#bddf26"]


#FIGURE 2 SUBPLOT
outputGrowth = make_subplots(
    rows=2, cols=1, subplot_titles=("A. Chassis concentration", "B. Substrate Flux"))

outputGrowth.add_trace(go.Scatter(x=t, y=BsF, line=dict(color = color[0]), name = "Chassis", showlegend=False), row=1, col=1)
outputGrowth.add_trace(go.Scatter(x=t, y=SubF, line=dict(color = color[3]),name = "Substrate", showlegend=False), row=2, col=1)

outputGrowth.update_xaxes(title_text="Time (min)", row=1, col=1)
outputGrowth.update_xaxes(title_text="Time (min)", row=2, col=1)
outputGrowth.update_yaxes(title_text='B. subtilis [Bs] (cell/L)', row=1, col=1)
outputGrowth.update_yaxes(title_text='Substrate [S] (g/ml)', row=2, col=1)

outputGrowth.update_layout(template='simple_white',
                  autosize=False,
    width=800,
    height=800,
    plot_bgcolor= "rgba(0, 0, 0, 0)",
  paper_bgcolor= "rgba(0, 0, 0, 0)")
outputGrowth.show()


outputGrowth.write_image("images/Growth_sim_UNILA_LatAm.png")
outputGrowth.write_html("images/Growth_sim_UNILA_LatAm.html")


#FIGURE 3 SUBPLOT
outputRNA = make_subplots(
    rows=2, cols=1, subplot_titles=("A. mRNA concentration", "B. mRNA Concentration (within 30min)"))

outputRNA.add_trace(go.Scatter(x=t, y=mRNAF, line=dict(color = color[1]),name = "mRNAtoehold"), 
                       row=1, col=1)
outputRNA.add_trace(go.Scatter(x=t, y=mRNAonF, line=dict(color = color[2]), name = "mRNAon"), 
                       row=2, col=1)
outputRNA.add_trace(go.Scatter(x=t[:30], y=mRNAF, line=dict(color = color[1]),name = "mRNAtoehold", showlegend=False), 
                       row=1, col=1)
outputRNA.add_trace(go.Scatter(x=t[:30], y=mRNAonF, line=dict(color = color[2]),name = "mRNAon",  showlegend=False), 
                       row=2, col=1)
outputRNA.add_vline(x=mRNAon_delay, line_width=3, line_dash="dash", line_color="black")

outputRNA.update_xaxes(title_text="Time (min)", row=1, col=1)
outputRNA.update_xaxes(title_text="Time (min)", row=2, col=1)
outputRNA.update_yaxes(title_text='[mRNA] (nM)', row=1, col=1)
outputRNA.update_yaxes(title_text='[mRNA] (nM)', row=2, col=1)

outputRNA.update_layout(template='simple_white',
                  autosize=False,
    width=800,
    height=800,
    plot_bgcolor= "rgba(0, 0, 0, 0)",
  paper_bgcolor= "rgba(0, 0, 0, 0)")


outputRNA.show()
outputRNA.write_image("images/RNA_sim_UNILA_LatAm.png")
outputRNA.write_html("images/RNA_sim_UNILA_LatAm.html")

#FIGURE 4 SUBPLOT

outputAPL = make_subplots(
    rows=2, cols=1, subplot_titles=("A. AMPs concentrations", "B. Leishmania viable cells (cell/L)"))

outputAPL.add_trace(go.Scatter(x=t, y=proAMPF, line=dict(color = color[0]),name = "proAMP"), 
                       row=1, col=1)
outputAPL.add_trace(go.Scatter(x=t, y=AMPaF, line=dict(color = color[1]), name = "AMPon"), 
                       row=1, col=1)
outputAPL.add_trace(go.Scatter(x=t, y=LeishF, line=dict(color = color[2]),name = "Leishmania"), 
                       row=2, col=1)

outputAPL.add_vline(x=Leishmin_delay, line_width=3, line_dash="dash", line_color="black")

outputAPL.update_xaxes(title_text="Time (min)", row=1, col=1)
outputAPL.update_xaxes(title_text="Time (min)", row=1, col=2)
outputAPL.update_yaxes(title_text='[peptide] (nM)', row=1, col=1)
outputAPL.update_yaxes(title_text='viable cells (cell/L)', row=1, col=2)

outputAPL.update_layout(template='simple_white',
                  autosize=False,
    width=800,
    height=800,
    plot_bgcolor= "rgba(0, 0, 0, 0)",
  paper_bgcolor= "rgba(0, 0, 0, 0)")


outputAPL.show()
outputAPL.write_image("images/AMPLeish_sim_UNILA_LatAm.png")
outputAPL.write_html("images/AMPLeish_sim_UNILA_LatAm.html")



# **GLOBAL SENSITIVITY ANALYSIS**

In [ ]:
pip install SALib

In [ ]:
from scipy import integrate as sp
import numpy as np
import SALib
from SALib.sample import saltelli
from SALib.analyze import sobol
#from MasterModel import *
                   #K05Bs, FD, trigger0, v, Ks, y, lamb, alphamax, sig, tmax, R, DEGrna, DEGamp, Kd, ktl, dig_rat, growthLeish, n
print([ K05Bs, FD,Trigger, v, Ks, y, lamb, alphamax, sig, tmax, R, DEGrna, DEGamp, Kd, ktl, dig_rat, growthLeish, n, Bsi, Si, mRNA,mRNAon, proAMP, AMPa, Leish1])
fram1 = np.array([ K05Bs, FD,Trigger, v, Ks, y, lamb, alphamax, sig, tmax, R, DEGrna, DEGamp, Kd, ktl, dig_rat, growthLeish, n, Bsi, Si, mRNA,mRNAon, proAMP, AMPa, Leish1])*0.95
fram2 = np.array([ K05Bs, FD,Trigger, v, Ks, y, lamb, alphamax, sig, tmax, R, DEGrna, DEGamp, Kd, ktl, dig_rat, growthLeish, n, Bsi, Si, mRNA,mRNAon, proAMP, AMPa, Leish1])*1.05
problem = {
    'num_vars': 25,  # a's, b's and initial condition
    'names': ['K05Bs', 'FD','Trigger','v', 'Ks', 'y', 'lamb', 'alphamax', 'sig', 'tmax', 'R', 'DEGrna', 'DEGamp', 'Kd', 'ktl', 'dig_rat', 'growthLeish', 'n',
              "Bs", "Si", 'mRNA', 'mRNAon', 'proAMP', 'AMPa', 'Leish'],
    'bounds': np.column_stack((fram1,fram2))}
N1=10000
#N1=10 
num_par=25
print(N1*(2*num_par+2))



vals = saltelli.sample(problem, N1)
Y = np.zeros([len(vals), 1])
Y = np.zeros([len(vals), 7])


Bs_X = pd.DataFrame()
Si_x = pd.DataFrame()
# print(Y)

for i in range(len(vals)):
    model = sp.odeint(modelkinetic, [vals[i][18], vals[i][19], vals[i][20], vals[i][21],
                                       vals[i][22], vals[i][23],vals[i][24]], t,
                        args=( vals[i][0], vals[i][1], vals[i][2], vals[i][3],
                        vals[i][4], vals[i][5], vals[i][6], vals[i][7], vals[i][8], vals[i][9],
                               vals[i][10], vals[i][11], vals[i][12], vals[i][13], vals[i][14], 
                              vals[i][15], vals[i][16], vals[i][17]))[len(ODE_Geral) - 1]
  
    Y[i][:] = model
    #Y[Y<=0,] = np.nextafter(0, 1)

    #Leish_minGSA.append(model[6].min())
    #Leish_delayGSA.append(t[np.nonzero(Leishv > Leishv[0]*0.5)[0][-1]])
    #Bs_GSA.append(Bsv)
    #Ls_GSA.append(Leishv)



Bs = sobol.analyze(problem, Y[:, 0], print_to_console=True, num_resamples=1000)
S = sobol.analyze(problem, Y[:, 1], print_to_console=True, num_resamples=1000)
AMP = sobol.analyze(problem, Y[:, 4], print_to_console=True, num_resamples=1000)
AMPon = sobol.analyze(problem, Y[:, 5], print_to_console=True, num_resamples=1000)
Leishm = sobol.analyze(problem, Y[:, 6], print_to_console=True, num_resamples=1000)

total_Bs, first_Bs, second_Bs =Bs.to_df()
total_S, first_S, second_S =S.to_df()
total_AMP, first_AMP, second_AMP = AMP.to_df()
total_AMPon, first_AMPon, second_AMPon = AMPon.to_df()
total_Leish, first_Leish, second_Leish = Leishm.to_df()




# Source: https://stackoverflow.com/questions/58504174/how-can-i-perform-sensitivity-analysis-on-odes-with-salib

ModuleNotFoundError: ignored

In [ ]:
from google.colab import files

export = [total_Bs, first_Bs, second_Bs,total_S, first_S, second_S,total_AMP, first_AMP, second_AMP,total_AMPon, 
          first_AMPon, second_AMPon ,total_Leish, first_Leish, second_Leish]
export1 = ["total_Bs", "first_Bs", "second_Bs","total_S", "first_S", "second_S","total_AMP", "first_AMP", "second_AMP","total_AMPon",
          "first_AMPon", "second_AMPon" ,"total_Leish", "first_Leish", "second_Leish"]

for i in range(len(export)):
  df = export[i]
  name = export1[i]+".csv"
  df.to_csv(name)
  files.download(name)

In [ ]:
from google.colab import files
import numpy
dataexp =["Bs", "S", "mRNA", "mRNAon", "AMP", "AMPon", "Leish"]

for i in range(len(dataexp)):
  df = np.asarray(Y[:,i])
  name = dataexp[i]+".csv"
  numpy.savetxt(name,df,delimiter=",")
  files.download(name)

# **ANALYSIS 3D Leishmania**
Figure 8

In [19]:
pip install colour

In [25]:
ran = 10
Bs_delay3D = []
Bs_delay3Df =[]
Leish_n3D = []
nr3D = []
vr3D = []
nrang = np.linspace(1, 4, 20)
vrang = np.linspace(0, 4*v, 20)
print(vrang)
print(nrang)
i = 0
j= 0
for vr in vrang:
  Leish_v3D = []
  i=i+1
  for nr in nrang:
    j=j+1
    args = (K05Bs, FD,Trigger,vr, Ks, y, lamb, alphamax, sig, tmax, R, DEGrna, DEGamp, Kd, ktl, dig_rat, growthLeish, nr)
    Bsn, sn, mRNAn, mRNAonn, proAMPn, AMPan, Leishn = odeint(modelkinetic, P0, t, args).transpose()
   

    Bs_delay3D.append(Bsn.min())
    Leish_v3D.append(t[np.nonzero(Leishn > Leish1*0.5)[0][-1]])
    #nr3D.append(nr)
    #vr3D.append(vr)
  Leish_n3D.append(Leish_v3D)
  Bs_delay3Df.append(Bs_delay3D)

print(np.asarray(Leish_n3D))
z = np.asarray(Leish_n3D)
x = np.asarray(vrang)
y = np.asarray(nrang)

[0.         0.00336842 0.00673684 0.01010526 0.01347368 0.01684211
 0.02021053 0.02357895 0.02694737 0.03031579 0.03368421 0.03705263
 0.04042105 0.04378947 0.04715789 0.05052632 0.05389474 0.05726316
 0.06063158 0.064     ]
[1.         1.15789474 1.31578947 1.47368421 1.63157895 1.78947368
 1.94736842 2.10526316 2.26315789 2.42105263 2.57894737 2.73684211
 2.89473684 3.05263158 3.21052632 3.36842105 3.52631579 3.68421053
 3.84210526 4.        ]
[[500. 500. 500. 500. 500. 500. 500. 500. 500. 500. 500. 500. 500. 500.
  500. 500. 500. 500. 500. 500.]
 [ 41.  63. 116. 500. 500. 500. 500. 500. 500. 500. 500. 500. 500. 500.
  500. 500. 500. 500. 500. 500.]
 [ 34.  47.  68. 107. 226. 500. 500. 500. 500. 500. 500. 500. 500. 500.
  500. 500. 500. 500. 500. 500.]
 [ 30.  41.  55.  76. 107. 155. 254. 445. 500. 500. 500. 500. 500. 500.
  500. 500. 500. 500. 500. 500.]
 [ 28.  37.  49.  63.  82. 106. 134. 164. 198. 247. 305. 346. 376. 399.
  419. 435. 449. 461. 472. 482.]
 [ 27.  35.  44.  56.  70

In [26]:
name = 'eye = (x:2, y:2, z:0.1)'
camera = dict(
    eye=dict(x=1.3, y=2, z=0.1)
)

output3D = go.Figure(data=[go.Surface(z=z, x=x, y=y)])
output3D.update_traces(showscale=False, selector=dict(type='surface'))
output3D.update_layout(scene = dict(
                    xaxis_title='v',
                    yaxis_title='n',
                    zaxis_title="Half-Elimination Response Time (min)"),
                  autosize=False,
                  width=650, height=650,
                  margin=dict(l=90, r=50, b=65, t=90),
                  template='simple_white',
                  #showscale = False,
                  scene_camera=camera,
                  plot_bgcolor= "rgba(0, 0, 0, 0)",
                  paper_bgcolor= "rgba(0, 0, 0, 0)")

output3D.show()
output3D.write_image("images/3D_sim_UNILA_LatAm.png")
output3D.write_html("images/3D_sim_UNILA_LatAm.html")